In [ ]:
%%html
<style>
table {float:left}
</style>

In this notebook, we used two datasets of pancreatic cells. For gene markers, we used the Panglao Cell Type Marker database.

We used the Baron (2016) dataset for training the neural network (NN) because it has more cells and classes, and we used the Xin (2016) dataset for testing purposes because all of its classes are present in the Baron dataset.

In [ ]:
EPOCHS = 85

In [ ]:
import saly
import saly.backend as S
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline


SPECIES = 'Human'
DATA_PATH = '/Users/MarkDavidLongar/Documents/Coding/MachineLearning/Data/'
DATASET_BARON_PATH = DATA_PATH + 'baron_2016h.h5ad' 
DATASET_XIN_PATH = DATA_PATH + 'xin_2016.h5ad'
MARKER_PATH = DATA_PATH + 'panglao_gene_markers.tab.gz'

# Load the data

In [ ]:
baron_data, baron_labels = saly.load_h5ad(DATASET_BARON_PATH)
xin_data, xin_labels = saly.load_h5ad(DATASET_XIN_PATH)

### Baron

In [ ]:
print(baron_data.shape)
baron_labels.unique()

Baron has 8569 cells and 20125 genes.

The cells are classified into 9 classes: Acinar cells, Alpha cells, Beta cells, Delta cells, Ductal cells, Endothelial cells, ***Other***, PaSC (i.e. Pancreatic stellate cells) and PP cells (i.e. Gamma cells).

### Xin

In [ ]:
print(xin_data.shape)
xin_labels.unique()

Xin has 1492 cells and 39851 genes.

The cells are classified into 4 classes: Alpha cells, Beta cells, Delta cells and PP cells (i.e. Gamma cells).

# Drop rows

We removed every cell classified as 'Other' from the Baron dataset.

In [ ]:
baron_data, baron_labels = saly.drop_rows(baron_data, baron_labels, ['Other'])

This removed 118 cells from Baron, leaving us with 8451 left.

# Load marker genes

We make sure to only load the Human marker genes.

In [ ]:
markers_db = saly.load_markers(MARKER_PATH, SPECIES)

#### Get the markers and only keep those found in both datasets

In [ ]:
markers = saly.get_mutual_markers([baron_data.columns, xin_data.columns], markers_db)

We do this because we want to train the network only on the marker genes found in both datasets, and drop the rest.

In [ ]:
#print(len(S.get_used_markers(baron_data.columns, markers_db)), len(S.get_used_markers(xin_data.columns, markers_db)))
print(len(markers))

Baron has 7618 markers, and Xin has 7610.

We used only those found in both datasets (7545).

*Note that in this context* ***Marker ≠ Marker Gene*** *because multiple cell types (i.e. markers) can have the same marker gene and so in this context, the number 7545 means 7545 connections in the partially-connected Marker Layer between the cell types and the genes.*

### Make sure every class is in the marker layer

Because some cell types are differently labelled in the marker database, we created a dictionary of aliases.

We also make sure to print out any label that cannot be found in either the list of markers or in the dictionary of aliases.

In [ ]:
marker_aliases = {'PaSC' : 'Pancreatic stellate cells',
                  'PP cells' : 'Gamma (PP) cells',
                  'Endothelial cell' : 'Endothelial cells'}

saly.check_labels([baron_labels, xin_labels], markers, marker_aliases)

# Drop unused genes

We used only the genes found in both datasets and the marker gene database, and therefore, we will connect with the partially-connected marker layer.

In [ ]:
baron_data = saly.drop_unused_genes(baron_data, markers)
xin_data = saly.drop_unused_genes(xin_data, markers)

After this operation we are left with 4173 genes.

#### Make sure the data sets' shapes match

In [ ]:
saly.check_shape([baron_data, xin_data])

# Data transformation

#### Preprocess the data

We shuffle, log10 transform and split the data into 70%, 15%, 15% splits.

In [ ]:
(train_x, train_y), (validation_x, validation_y), (test_x, test_y) = saly.preprocess_data(baron_data, baron_labels,
                                                                                          train=0.7, validation=0.15,
                                                                                          test=0.15)

#### Log10 transform the Xin data, too

In [ ]:
xin_data = S.log_10(xin_data)

# Building the model

For the model we define a partially-connected Marker Layer.

The layer has a node for each cell type whose genes are present in both data sets.
In this case, there are 179 nodes in the marker layer representing 179 cell types. 

We only connect nodes (i.e. the cell types) in the marker layer to their marker genes at the input.

We constructed the partially connected marker layer by taking a dense layer and adding a connection mask (a binary matrix) to it, which we multiply with the output matrix.

We trained the network for 100 epochs.

### The Architecture
|Layer|Nodes|
|-----|------|
|Input|4173 (genes)|
|Markers|179 (cell types)|
|Dense layer|100|
|**Bottleneck**|**25**|
|Dense layer|100|
|*Dropout*|*10%*|
|Output|4173 (genes)|

#### Create the model

In [ ]:
model, marker_model, encoder_model = saly.build_model(baron_data, markers);

In [ ]:
model.summary()

# Train the model

In [ ]:
history = saly.train_model(model, train_x, EPOCHS, validation_data=validation_x, verbose=0);

In [ ]:
saly.plot_model_history(history)
print("Test loss: ", saly.test_model(model, test_x))

# Data analysis

In [ ]:
baron_cell_activations = marker_model.predict(test_x)

In [ ]:
saly.plot_activation_distribution(baron_cell_activations, markers)

In [ ]:
saly.get_results(test_y, baron_cell_activations, markers, marker_aliases)

**Mreža pravilno klasificira 82% testnih podatkov (iz baze na kateri se je učila)**

Spodnji graf prikazuje aktivacije tipov celic. Razvidno je, da izstopa samo en celični tip, ostali tipi pa izrecno manj.

### New data

**Mreža pravilno klasificira 86% čisto novih podatkov iz druge baze**

In [ ]:
xin_cell_activations = marker_model.predict(xin_data)

In [ ]:
saly.plot_activation_distribution(xin_cell_activations, markers)

In [ ]:
saly.get_results(xin_labels, xin_cell_activations, markers, marker_aliases)

## Graphs

In [ ]:
import random

def get_random_color(pastel_factor = 0.5):
    pastel_factor = random.uniform(0, 1.0)
    return [(x+pastel_factor)/(1.0+pastel_factor) for x in [random.uniform(0,1.0) for i in [1,2,3]]]

colours = {}
for label in graph_labels.unique():
    colour = (get_random_color())
    colours.update({label:colour})
    
fig = plt.figure()
ax = fig.add_subplot(111)

x = list(range(0, len(colours.values()) * 2, 2))
y = [0]*len(x)
c = list(colours.values())

ax.scatter(x,y,s=400,c=c)
ax.set_title('Plot with Different Marker Color, matplotlib and plotly')

plt.show()

In [ ]:
import pickle

pickle_in = open("Data/Marker_Colours.pickle","rb")
colours = pickle.load(pickle_in)

In [ ]:
pickle_out = open("Data/Marker_Colours.pickle","wb")
pickle.dump(colours, pickle_out)
pickle_out.close()

In [ ]:
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from collections import OrderedDict
import operator

tnse = TSNE()
pca = PCA()
    
def draw_graph(x, y, model, colours, graph_name=''):
    tsne_out = model.fit_transform(x)
    
    plt.clf()
    plt.figure(figsize=(8, 8), dpi=80, facecolor='w', edgecolor='k')

    for i, entry in enumerate(tsne_out):
        plt.scatter(entry[0], entry[1], color=colours[y.iloc[i]], label=y.iloc[i])
    
    handles, plt_labels = plt.gca().get_legend_handles_labels()
    by_label = OrderedDict(zip(plt_labels, handles))
    by_label = OrderedDict(sorted(by_label.items()))

    plt.legend(by_label.values(), by_label.keys(), bbox_to_anchor=(0., 1, 0, .1))

    plt.title(graph_name)
    plt.axis('off')
    plt.show()

In [ ]:
draw_graph(cell_activations, graph_labels, tnse, colours, graph_name='TSNE of marker activations')

# New Data

In [ ]:
draw_graph(new_cell_activations, new_graph_labels, tnse, colours, graph_name='TSNE of marker activations')

# Combined data

In [ ]:
def draw_comparison(x, y, new_alpha = 1.0, preloaded_tsne=None, title=''):
    if preloaded_tsne is None:
        tsne_out = tnse.fit_transform(x)
    else:
        tsne_out = preloaded_tsne

    plt.clf()
    plt.figure(figsize=(8, 8), dpi=80, facecolor='w', edgecolor='k')

    for i, entry in enumerate(tsne_out):
        colour = colours[y.iloc[i]]
        alpha = 1.0
        marker = '.'
        if(i <= test_x.shape[0]):
            alpha = new_alpha
            marker = 'x'
        plt.scatter(entry[0], entry[1], color=colour, label=y.iloc[i], alpha=alpha, marker=marker)

    handles, plt_labels = plt.gca().get_legend_handles_labels()
    by_label = OrderedDict(zip(plt_labels, handles))
    by_label = OrderedDict(sorted(by_label.items()))

    plt.legend(by_label.values(), by_label.keys(), bbox_to_anchor=(0., 1, 0, .1))

    plt.title(title)
    plt.axis('off')
    plt.show()
    
    #return tsne_out

**Na spodnjem grafu lahko opaziš, da se batch effect še vedno opazi, tudi na aktivacijah markerjev**

In [ ]:
x = test_x.append(new_data_x)
y = test_y.append(new_labels)
compare_cell_activations = encoder_model.predict(x)
tsne_out = draw_comparison(compare_cell_activations, y, new_alpha=0.2, title='Combined TSNE of marker activations')

In [ ]:
draw_comparison(np.vstack((top3, new_top3)), y, new_alpha=0.2, title='Combined TSNE of top 3 activations')

# Reconstruction

**Tukaj sem vzel nove podatke in jih vrgel v avtoenkoder.**

**Nato sem vzel izhodne podatke avtoenkoderja ter vzel njihove aktivacije sloja markerjev.**

**Tako sem dobil nove rezultate, ki so bili konsistentno boljši. V tem primeru za 11%.**

Moja hipoteza je, da se tako delno znebimo batch effecta in zato dobimo boljše rezultate.

In [ ]:
reconstruction = model.predict(new_data_x)
reconstruction = pd.DataFrame(reconstruction, columns=new_data_x.columns)
reconstructed_cell_activations = marker_model.predict(reconstruction)

In [ ]:
draw_graph(reconstructed_cell_activations, new_graph_labels, tnse, colours,
           graph_name='TSNE of reconstructed marker activations')

In [ ]:
_ = get_results(cell_types, reconstructed_cell_activations, new_graph_labels)

Tudi pri teh podatkih batch effect še vedno ostane.

In [ ]:
x = test_x.append(reconstruction)
y = test_y.append(new_labels)
compare_cell_activations = encoder_model.predict(x)
tsne_out = draw_comparison(compare_cell_activations, y, new_alpha=0.2, title='Combined TSNE of marker activations')